# Web Scraping: EveryNoise.com

source: [http://everynoise.com/everynoise1d.cgi?scope=all]



In [1]:
from bs4 import BeautifulSoup
import requests, sys
from collections import defaultdict
from multiprocessing.pool import ThreadPool
from tqdm import tqdm
import json

In [2]:
## Request HTML

r = requests.get('http://everynoise.com/everynoise1d.cgi?vector=popularity&scope=all')

In [3]:
## Parse Genre HTML Elements
soup = BeautifulSoup(r.text, "html.parser")
table = soup.body.table
elements = table.find_all('td', 'note')
genres = []
for i, row in enumerate(elements):
    if i % 2:
        genres.append(row.contents[0].contents[0])
print("Total genres:", len(genres))
print("First ten genres:\n", genres[:10])

Total genres: 5071
First ten genres:
 ['pop', 'dance pop', 'rap', 'pop dance', 'pop rap', 'rock', 'post-teen pop', 'latin', 'hip hop', 'trap']


In [4]:
## Request Playlist HTMLs and extract link

def requestPlaylists(genres, links, threadName):
    spotifyLinks = dict()
    with tqdm(total=len(genres), position=1, desc=threadName) as pbar:
        for i in range(len(genres)):
            playlist_r = requests.get('http://everynoise.com/everynoise1d.cgi?root={}&scope=all'.format(genres[i]))
            soup = BeautifulSoup(playlist_r.text, "html.parser")
            playlistLink = soup.find_all(id='spotifylink')
            spotifyLinks[genres[i]] = playlistLink[0]['href']
            if(i > 0 and (i+1) % 50 == 0):
                pbar.update(50)
            elif(i == len(genres) - 1):
                pbar.update(len(genres) % 50)
    links.update(spotifyLinks)
    print(threadName, ' completed.')

In [5]:
## Split genres into chunks

chunk_indices = []

for i in range(0, len(genres) - len(genres) % 1000, 1000):
    chunk_indices.append((i, i + 1000))

chunk_indices.append((len(genres) - len(genres) % 1000, len(genres)))


In [6]:
## Create a thread for each chunk to fetch links

links = dict()

pool = ThreadPool(len(chunk_indices))

for t in range(len(chunk_indices)):
    pool.apply_async(requestPlaylists, args=(genres[chunk_indices[t][0]: chunk_indices[t][1]], links, "Thread {}".format(t+1)))

pool.close()
pool.join()

## Been trying to make each progress bar stay on its line


Thread 6: 100%|██████████| 71/71 [10:13<00:00,  8.64s/it]
Thread 6  completed.

Thread 3: 100%|██████████| 1000/1000 [1:57:08<00:00,  7.03s/it]
Thread 3  completed.

Thread 4: 100%|██████████| 1000/1000 [1:58:31<00:00,  7.11s/it]
Thread 4  completed.

Thread 5: 100%|██████████| 1000/1000 [1:59:31<00:00,  7.17s/it]
Thread 5  completed.

Thread 1: 100%|██████████| 1000/1000 [2:02:17<00:00,  7.34s/it]
Thread 1  completed.

Thread 2: 100%|██████████| 1000/1000 [2:02:27<00:00,  7.35s/it]Thread 2  completed.



In [7]:
print(len(links))

5071


In [10]:
with open('../data/spotify_links.json', 'w') as outfile:
    json.dump(links, outfile, indent=4)